In [6]:
import os, sys

import sgpp, sgml, dproc, sgutil
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print(sys.version)

from sklearn.pipeline import make_pipeline

3.12.6 (main, Sep 30 2024, 02:19:13) [GCC 9.4.0]


In [7]:
sc = sgutil.SGCache('img', 'result', 'model')
p = make_pipeline(
    sgpp.PolarsProcessor(),
    sgpp.ExprProcessor({
        'ELm_num': (pl.col('Episode_Length_minutes').fill_null(pl.col('Episode_Length_minutes').mean()).clip(5, 120) - 5) / 115,
        'GP': (pl.col('Guest_Popularity_percentage').fill_null(pl.col('Guest_Popularity_percentage').clip(0,100).mean()).clip(0, 100) / 100),
        'HP': pl.col('Host_Popularity_percentage').clip(0, 100) / 100,
        'NAd': (pl.when(pl.col('Number_of_Ads') > 4).then(0).otherwise(pl.col('Number_of_Ads'))).fill_null(0.0) /3 ,
        'Number_of_Ads': (pl.when(pl.col('Number_of_Ads') > 4).then(0).otherwise(pl.col('Number_of_Ads'))).fill_null(0).cast(pl.Int8),
        'ELm_na': pl.col('Episode_Length_minutes').is_null(),
        'GPp_na': pl.col('Guest_Popularity_percentage').is_null(),
        'ELm_sqrt': ((pl.col('Episode_Length_minutes').fill_null(pl.col('Episode_Length_minutes').mean()).clip(5, 120) - 5) / 115)  ** 0.5
    }),
    sgpp.PandasConverter(index_col = 'id'),
)
p.fit(['data/train.csv'])
df_train = p.transform(['data/train.csv'])
df_test = p.transform(['data/test.csv'])

In [10]:
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import KFold, cross_validate, cross_val_score, ShuffleSplit
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector
kf = KFold(5, shuffle = True, random_state = 123)
ss = ShuffleSplit(1, train_size = 0.8, random_state = 123)
target = 'Listening_Time_minutes'
X_all = [i for i in df_train.columns if i != target]

def get_validation_splitter(validation_fraction):
    return lambda x: train_test_split(x, test_size = validation_fraction)

config = {
    'predict_func': lambda m, df, X: pd.Series(m.predict(df[X]), index = df.index),
    'score_func': lambda df, prds: root_mean_squared_error(df[target], prds),
    'validation_splitter': get_validation_splitter,
    'progress_callback': sgml.ProgressCallBack(), 
    'return_train_scores': True,
    'y': target,
}

lr_adapter = sgml.SklearnAdapter(LinearRegression)
lgb_adapter = sgml.LGBMAdapter(lgb.LGBMRegressor, progress = 50)
xgb_adapter = sgml.XGBAdapter(xgb.XGBRegressor, progress = 50)

2.Feature Engineering.ipynb 에서 발견한 속성들로 학습을 시킵니다.

In [21]:
X_tgt_sel, _ = sc.read_result('tgt_sel')

In [31]:
from sklearn.preprocessing import TargetEncoder
def tge_proc(df_train, df_valid):
    X_tgt_var = pd.Series(X_tgt_sel).str.split(',').explode().unique().tolist()
    df_train_all, df_valid_all = list(), list()
    tge = make_pipeline(
        sgpp.CatCombiner2(pd.Series(X_tgt_sel).str.split(',').tolist()), TargetEncoder()
    ).set_output(transform = 'pandas')
    return tge.fit_transform(df_train[X_tgt_var], df_train[target]), tge.transform(df_valid[X_tgt_var])

for i, (train_idx, valid_idx) in enumerate(kf.split(df_train[X_all], df_train[target])):
    print("generating {} Fold".format(i))
    sc.cache_result(
        'kf_sp_{}2'.format(i),
        lambda : tge_proc(df_train.iloc[train_idx], df_train.iloc[valid_idx]), rerun = 0
    )
X_tgt = X_tgt_sel

generating 0 Fold
generating 1 Fold
generating 2 Fold
generating 3 Fold
generating 4 Fold


# Linear Regression 2

In [32]:
hparams = {
    'X_num': ['GP', 'HP', 'NAd', 'ELm_sqrt', 'ELm_num', 'ELm_na'] + X_tgt
}
rmse_oof = list()
for i, (train_idx, valid_idx) in enumerate(kf.split(df_train[X_all], df_train[target])):
    df_tgt_train, df_tgt_valid = sc.cache_result(
        'kf_sp_{}2'.format(i),
        lambda : tge_proc(df_train.iloc[train_idx], df_train.iloc[valid_idx])
    )
    result = sgml.train(
        dproc.join_and_assign(df_train.iloc[train_idx], df_tgt_train), hparams, config, lr_adapter
    )
    result = result[0]
    if 'preprocessor' in result:
        m = make_pipeline(result['preprocessor'], result['model'])
    else:
        m = result['model']
    rmse_oof.append(
        root_mean_squared_error(
            df_train.iloc[valid_idx][target],
            m.predict(
                dproc.join_and_assign(df_train.iloc[valid_idx], df_tgt_valid)[result['variables']]
            )
        )
    )
rmse_oof

[np.float64(12.80809672753578),
 np.float64(12.831797164631581),
 np.float64(12.869177151735707),
 np.float64(12.89695525439413),
 np.float64(12.912815772222732)]

# LGB 2

In [33]:
hparams = {
    'model_params': {'num_leaves': 1024, 'n_estimators': 1500, 'learning_rate': 0.01, 'colsample_bytree': 0.7},
    'X_num': ['GP', 'HP', 'NAd', 'ELm_num', 'ELm_na'] + [i.replace(',', '_') for i in X_tgt],
    'metric': 'rmse'
}
rmse_oof = list()
for i, (train_idx, valid_idx) in enumerate(kf.split(df_train[X_all], df_train[target])):
    df_tgt_train, df_tgt_valid = sc.read_result('kf_sp_{}2'.format(i))
    df_tgt_train, df_tgt_valid = df_tgt_train.rename(columns = lambda x: x.replace(',', '_')), df_tgt_valid.rename(columns = lambda x: x.replace(',', '_'))
    result = sgml.train(
        dproc.join_and_assign(df_train.iloc[train_idx], df_tgt_train), hparams, config, lgb_adapter
    )
    result = result[0]
    if 'preprocessor' in result:
        m = make_pipeline(result['preprocessor'], result['model'])
    else:
        m = result['model']
    rmse_oof.append(
        root_mean_squared_error(
            df_train.iloc[valid_idx][target],
            m.predict(
                dproc.join_and_assign(df_train.iloc[valid_idx], df_tgt_valid)[result['variables']]
            )
        )
    )

Round:   0%|          | 0/1500 [00:00<?, ?it/s]

Round:   0%|          | 0/1500 [00:00<?, ?it/s]

Round:   0%|          | 0/1500 [00:00<?, ?it/s]

Round:   0%|          | 0/1500 [00:00<?, ?it/s]

Round:   0%|          | 0/1500 [00:00<?, ?it/s]

In [34]:
rmse_oof

[np.float64(11.8728253373632),
 np.float64(11.893152483420323),
 np.float64(11.898166553175242),
 np.float64(11.977559155461316),
 np.float64(11.984174012061468)]